In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst_efd_client import EfdClient

In [ ]:
def mountSpeed(az, el):
    # Calculate theoretical drive speed vs Az/El
    t0 = Time('2023-01-01T00:00:00') # doesn't really matter when
    altAz0 = SkyCoord(AltAz(alt=el*u.deg, az=az*u.deg, obstime=t0, location=AUXTEL_LOCATION))
    RaDec = altAz0.transform_to(ICRS)
    t1 = t0 + TimeDelta(1.0, format='sec')
    altAz1 = RaDec.transform_to(AltAz(obstime=t1, location=AUXTEL_LOCATION))
    #print(altAz1)
    newAz = altAz1.az.deg
    if az < 5.0 and newAz > 355.0:
        delta = newAz -360.0 - az
    else:
        delta = newAz - az

    azVel = abs(delta) * 3600.0
    elVel = abs(altAz1.alt.deg - el) * 3600.0
    #print(az, altAz1.az.deg, el, altAz1.alt.deg)
    return(azVel, elVel)
Naz = 100
Nel = 50
azimuths = np.linspace(0, 360, Naz)
els = np.linspace(5, 85, Nel)
azValues = np.zeros([Naz, Nel])
elValues = np.zeros([Naz, Nel])
for i, az in enumerate(azimuths):
    for j, el in enumerate(els):
        [azSpeed, elSpeed] = mountSpeed(az, el)
        azValues[i,j] = azSpeed
        elValues[i,j] = elSpeed
        #print(az, el, mountSpeed(az, el)[0])
    
r, theta = np.meshgrid(els, np.radians(azimuths))


In [ ]:
#-- Plot... ------------------------------------------------
fig = plt.figure(figsize=(10,8))
plt.subplots_adjust(wspace = 1.0)
vmin1 = -2.0
vmax1 = 2.0
levels1 = np.linspace(vmin1, vmax1, 21)
vmin2 = 0.0
vmax2 = 15.0
levels2 = np.linspace(vmin2, vmax2, 16)
ax1 = plt.subplot(121, projection='polar')
ax1.set_title("Log10 Theoretical Azimuth speed (arcseconds/sec)", y=1.1)
ax1.invert_yaxis()
contourf_1 = ax1.contourf(theta, r, np.log10(azValues), levels=levels1)
cax1 = fig.add_axes([ax1.get_position().x1+0.01,ax1.get_position().y0,0.02,ax1.get_position().height])
plt.colorbar(contourf_1, cax=cax1)
ax1.scatter([np.pi], [-AUXTEL_LOCATION.lat.deg], color='cyan', marker='x', s=200, label="SCP")
#ax1.legend()
ax2 = plt.subplot(122, projection='polar')
ax2.set_title("Theoretical Elevation speed (arcseconds/sec)", y=1.1)
ax2.invert_yaxis()
contourf_2 = ax2.contourf(theta, r, elValues, levels=levels2)
cax2 = fig.add_axes([ax2.get_position().x1+0.01,ax2.get_position().y0,0.02,ax2.get_position().height])
plt.colorbar(contourf_2, cax=cax2)
ax2.scatter([np.pi], [-AUXTEL_LOCATION.lat.deg], color='cyan', marker='x', s=200, label="SCP")
#ax2.legend()
plt.savefig('/home/cslage/DATA/Tracking_Speeds_11Jun25.png')